In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('./trainclauses.csv')
df.head()
print(df)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

                                                 Text Label
0   1. Limitation of Liability: Do not agree to a ...   Bad
1   2. Pay-when/if-paid requirement in a flow down...   Bad
2   a. Where contract client is in long-standing g...   Bad
3   b. For other architects/contractors/primes whe...   Bad
4   c. For other architects/contractors/primes whe...   Bad
5   a. If the client doesn’t agree, we escalate to...   Bad
6           3. Limitations on Working for Competitors   Bad
7   a. Always push back, propose accommodating lan...   Bad
8   4. Payment Terms turnaround time that is beyon...   Bad
9   a. Up to 45 days payment terms are acceptable ...   Bad
10  b. Greater than 45 days must get chief financi...   Bad
11  · Chief financial office to consult chief reve...   Bad
12                               5. Background Checks   Bad
13  a. Use the client’s provider. Review the clien...   Bad
14  · If necessary, escalate to chief financial of...   Bad
15  6. Non-solicitation - Refusal to agr

In [36]:
def process_data(row):

    text = row['Text']
    text = str(text)
    text = ' '.join(text.split())

    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    label = 0
    if row['Label'] == 'Good':
        label += 1

    encodings['label'] = label
    encodings['text'] = text

    return encodings
processed_data = []
#print(df)
for i in range(len(df[:1000])):
    processed_data.append(process_data(df.iloc[i]))
print(len(processed_data))
new_df = pd.DataFrame(processed_data)

train_df, valid_df = train_test_split(
    new_df,
    test_size=0.2,
    random_state=2022
)


51


In [ ]:
!pip install Dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.29
    Uninstalling SQLAlchemy-2.0.29:
      Successfully uninstalled SQLAlchemy-2.0.29
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.52 which is incompatible.


In [ ]:
!pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00


In [37]:
import pyarrow as pa
from datasets import Dataset

train_hg = Dataset(pa.Table.from_pandas(train_df))
valid_hg = Dataset(pa.Table.from_pandas(valid_df))

In [38]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [39]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="./result", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hg,
    eval_dataset=valid_hg,
    tokenizer=tokenizer
)

In [40]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,0.492252
2,No log,0.470348
3,No log,0.337617


TrainOutput(global_step=15, training_loss=0.3443155288696289, metrics={'train_runtime': 183.734, 'train_samples_per_second': 0.653, 'train_steps_per_second': 0.082, 'total_flos': 7893331660800.0, 'train_loss': 0.3443155288696289, 'epoch': 3.0})

In [41]:
trainer.evaluate()

{'eval_loss': 0.33761662244796753,
 'eval_runtime': 4.0584,
 'eval_samples_per_second': 2.71,
 'eval_steps_per_second': 0.493,
 'epoch': 3.0}

In [42]:
model.save_pretrained('./model/')

In [43]:
from transformers import AutoModelForSequenceClassification
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

new_model = AutoModelForSequenceClassification.from_pretrained('./model/').to(device)

In [44]:
from transformers import AutoTokenizer

new_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

In [45]:
import torch
import numpy as np

def get_prediction(text):
    encoding = new_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = new_model(**encoding)

    logits = outputs.logits
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    sigmoid = torch.nn.Sigmoid()
    print(sigmoid)
    probs = sigmoid(logits.squeeze().cpu())
    probs = probs.detach().numpy()
    label = np.argmax(probs, axis=-1)

    if label == 1:
        return {
            'tag': 'Good',
            'probability': probs[1]
        }
    else:
        return {
            'tag': 'Bad',
            'probability': probs[0]
        }

In [50]:
get_prediction("The parties are independent contractors, and nothing contained in this Agreement will be construed as creating any agency, partnership, joint venture or other form of joint enterprise, employment, or fiduciary relationship between them.")

Sigmoid()


{'tag': 'Good', 'probability': 0.5504104}

In [12]:
!pip install torch